In [110]:
import pandas as pd
import glob
import os
from sqlalchemy import create_engine

os.listdir("bases/")

['residencias.csv',
 'media_preco.csv',
 'airbnb_ny_2019.csv',
 'mapeamento_vizinhanca.csv']

In [111]:
airbnb = pd.read_csv("bases/airbnb_ny_2019.csv")

In [112]:
mapviz = pd.read_csv('bases/mapeamento_vizinhanca.csv', sep=';')

In [113]:
f_airbnb = airbnb.drop_duplicates()

In [114]:
f_airbnb = f_airbnb.dropna()

In [115]:
f_mapviz = mapviz.drop_duplicates()

In [116]:
f_mapviz = f_mapviz.dropna()

In [117]:
new_airbnb = pd.merge(left=f_airbnb, right=f_mapviz, how='left', left_on='neighbourhood', right_on='vizinhanca')

In [118]:
del new_airbnb['vizinhanca']

In [119]:
new_airbnb.rename(columns={"vizinhanca_grupo":"neighbourhood_group"}, inplace=True)

In [120]:
res = new_airbnb[new_airbnb.neighbourhood_group.isin(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island'])]

In [121]:
res.to_csv('bases/residencias.csv', index=False)

In [122]:
res = pd.read_csv('bases/residencias.csv', usecols=['neighbourhood_group', 'room_type','price'])

In [123]:
means = res.groupby(['neighbourhood_group', 'room_type']).mean()

In [124]:
means.to_csv('bases/media_preco.csv')

---
*A partir daqui, ocorre a carga de um Banco de dados SQLite para uso da API*

---

In [125]:
res = pd.read_csv('bases/residencias.csv')

In [126]:
engine = create_engine('sqlite:///titau/tItau.db', echo=True)

In [127]:
dbconn = engine.connect()

2020-08-03 17:49:58,728 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-03 17:49:58,730 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 17:49:58,732 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-03 17:49:58,734 INFO sqlalchemy.engine.base.Engine ()


In [128]:
res.to_sql('residencias', dbconn, if_exists='fail', index=False)

2020-08-03 17:49:58,974 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("residencias")
2020-08-03 17:49:58,976 INFO sqlalchemy.engine.base.Engine ()


ValueError: Table 'residencias' already exists.

In [129]:
means = pd.read_csv('bases/media_preco.csv')

In [130]:
means.to_sql('medias', dbconn, if_exists='fail',  index_label='id')

2020-08-03 17:51:24,058 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("medias")
2020-08-03 17:51:24,065 INFO sqlalchemy.engine.base.Engine ()


ValueError: Table 'medias' already exists.

In [131]:
sql_likes = '''
CREATE TABLE likes (
	id BIGINT, 
	residencia_id BIGINT,
    like INT default 1
)
'''
dbconn.execute(sql_likes)

2020-08-03 17:51:31,566 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE likes (
	id BIGINT, 
	residencia_id BIGINT,
    like INT default 1
)

2020-08-03 17:51:31,570 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 17:51:31,572 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) table likes already exists
[SQL: 
CREATE TABLE likes (
	id BIGINT, 
	residencia_id BIGINT,
    like INT default 1
)
]
(Background on this error at: http://sqlalche.me/e/13/e3q8)